# Functional Preprocessing

This notebooks preprocesses functional MRI images by executing the following processing steps:

1. Reorient Images to RAS
1. Removal of non-steady state volumes 
1. Motion Correction with SPM
1. Slice-wise Correction with SPM
1. Brain Extraction with SPM and FSL
1. Temporal Filter with Nilearn
1. Two- step coregistration using BBR with FSL, using WM segmentation from SPM
1. Spatial Filter (i.e. smoothing) with Nilearn

Additional, this workflow also performs:
 - Computes Friston's 24-paramter model for motion parameters
 - Computes Framewise Displacement (FD) and DVARS
 - Computes average signal in total volume, in GM, in WM and in CSF
 - Computes anatomical CompCor Components
 - Computes temporal CompCor Components
 
**Note:** This notebook requires that the anatomical preprocessing pipeline was already executed and that it's output can be found in the dataset folder under `dataset/derivatives/fmriflows/preproc_anat`. 

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format:

    dataset
    ├── analysis-func_specs.json
    ├── sub-{sub_id}
    │   └── func
    │       └── sub-{sub_id}_*task-{task_id}_run-{run_id}_bold.nii.gz
    └── task-{task_id}_bold.json
    
**Note:** Subfolders for individual scan sessions are optional. `fmriflows` will run the preprocessing on all files of a subject.

## Execution Specifications

This notebook will extract the relevant processing specifications from the `analysis-func_specs.json` file in the dataset folder. In the current setup, they are as follows:

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'analysis-func_specs.json')

with open(spec_file) as f:
    specs = json.load(f)

specs

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject names
subject_list = specs['subject_list']

# List of task names
task_list = specs['task_list']

# Mode and width of spatial filter, i.e. Low-Pass, fwhm of 6mm = ['LP', 6]
filters_spatial = specs['filters_spatial']

# High and low-pass filter to apply, i.e. Low-Pass of 100Hz = ["None", 100]
filters_temporal = specs['filters_temporal']

# Requested isometric voxel resolution after coregistration
voxel_res = specs['voxel_res']

# Reference time point (in ms) required for slice wise correction
ref_time = specs['ref_timepoint']

# Number of components to extract with ACompCor and TCompCor
ncomp = specs['n_confound_comp']

# Thresholds to use for outlier detection
outlier_thr = specs['outlier_thresholds']

# Number of cores to use
n_proc = specs['n_parallel_jobs']

# Cerating the Workflow

To ensure a good overview of the functional preprocessing, the workflow was divided into three subworkflows:

1. The Main Workflow, i.e. doing the actual preprocessing
2. The Confound Workflow, i.e. computing confound variables
3. Visualization Workflow, i.e. visualizating relevant steps for quality control

## Import Modules

In [ ]:
import os
import numpy as np
from os.path import join as opj
from nipype import Workflow, Node, MapNode, IdentityInterface, Function
from nipype.interfaces.image import Reorient
from nipype.interfaces.spm import SliceTiming, Realign
from nipype.interfaces.fsl import FLIRT, MeanImage, BET, BinaryMaths, ExtractROI
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.algorithms.confounds import (ACompCor, TCompCor, NonSteadyStateDetector,
                                         FramewiseDisplacement, ComputeDVARS)

In [ ]:
# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-dev/spm12_mcr/spm/spm12')

## Relevant Execution Variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/workingdir'

## Create a subworkflow for the Main Workflow

### Implement Nodes

In [ ]:
# Reorient anatomical images to RAS
reorient = Node(Reorient(orientation='RAS'), name='reorient')

In [ ]:
# Detection of Non-Steady State volumes
nonsteady_detection = Node(NonSteadyStateDetector(), name='nonsteady_detection')

In [ ]:
# Store number of non-steady state volumes in text file
def write_to_txt(in_file, n_volumes):
    
    import numpy as np
    out_file = in_file.replace('.nii.gz', '_nss.txt')
    np.savetxt(out_file, [n_volumes], fmt='%d')
    return out_file

write_nss = Node(Function(input_names=['in_file', 'n_volumes'],
                          output_names=['out_file'],
                          function=write_to_txt),
                 name='write_nss')

In [ ]:
# Removal of Non-Steady State volumes
nonsteady_removal = Node(ExtractROI(output_type='NIFTI',
                                    t_size=-1),
                         name='nonsteady_removal')

In [ ]:
# Extract sequence specifications of functional images
def get_parameters(run_id):
    
    import json
    import numpy as np
    from os.path import join as opj
    
    task_id = run_id.split('_run')[0][5:]
        
    func_desc = opj('/data', 'task-%s_bold.json' % task_id)

    with open(func_desc) as f:
        func_desc = json.load(f)

    # Read out relevant parameters
    TR = func_desc['RepetitionTime']
    slice_order = func_desc['SliceTiming']
    nslices = len(slice_order)
    time_acquisition = float(TR)-(TR/nslices)
    
    return TR, slice_order, nslices, time_acquisition

getParam = Node(Function(input_names=['run_id'],
                         output_names=['TR', 'slice_order',
                                       'nslices', 'time_acquisition'],
                         function=get_parameters),
                name='getParam')

In [ ]:
# Correct for motion
realign = Node(Realign(register_to_mean=True), name='realign')

In [ ]:
# Correct for slice-wise acquisition
slicetime = Node(SliceTiming(ref_slice=ref_time), name='slicetime')

In [ ]:
# Reset TR value after SPM's slice time correction
def add_TR_to_file(in_file, TR):
    
    import nibabel as nb
    
    # Load image
    img = nb.load(in_file)
    
    # Reset TR
    img.header.set_zooms(list(img.header.get_zooms()[:3]) + [TR])
    
    # Save file
    out_file = in_file.replace('.nii', '_TR.nii')
    img.to_filename(out_file)
    del img
    
    return out_file

reset_TR = Node(Function(input_names=['in_file', 'TR'],
                         output_names=['out_file'],
                         function=add_TR_to_file),
                name='reset_TR')

In [ ]:
# Remove skull signal from functional images
bet_func = Node(BET(functional=True,
                    mask=True,
                    output_type='NIFTI_GZ'),
                name='bet_func')

In [ ]:
# Computes mean image before coregistration
bet_mean = Node(MeanImage(dimension='T',
                          output_type='NIFTI_GZ'),
                name='bet_mean')

In [ ]:
# Pre-alignment of functional images to anatomical image
coreg_pre = Node(FLIRT(dof=6,
                       output_type='NIFTI_GZ'),
                 name='coreg_pre')

In [ ]:
# Coregistration of functional images to anatomical image with BBR
# using WM segmentation
coreg_bbr = Node(FLIRT(dof=6,
                       cost='bbr',
                       schedule=opj(os.getenv('FSLDIR'),
                                    'etc/flirtsch/bbr.sch'),
                       output_type='NIFTI_GZ'),
                 name='coreg_bbr')

In [ ]:
# Apply coregistration warp to functional images
applycoreg = Node(FLIRT(interp='spline',
                        apply_isoxfm=voxel_res,
                        datatype='short',
                        output_type='NIFTI_GZ'),
                 name='applycoreg')

In [ ]:
# Apply Temporal Filter
def apply_temporal_filter(in_file, TR, tFilter):
    
    import nibabel as nb
    from nilearn.image import clean_img, mean_img, math_img
    
    # Transform cutoff values into HZ
    low_pass, high_pass = tFilter
    postfix = 'tfilt_%s.%s' % (low_pass, high_pass)
    low_pass = 1. / low_pass if low_pass != 'None' else None
    high_pass = 1. / high_pass if high_pass != 'None' else None
    
    out_file = in_file.replace('.nii', '_%s.nii' % postfix)
    
    # Apply temporal filter and store it in new file
    img = clean_img(in_file, detrend=False, standardize=False, t_r=TR,
                    ensure_finite=True, low_pass=low_pass, high_pass=high_pass)
    affine = img.affine
    header = img.header

    # Add mean if image was high pass filtered
    if high_pass:
        img = math_img("img1 + img2[...,None]", img1=img, img2=mean_img(in_file))

    # Save temporal filtered image
    nb.Nifti1Image(img.get_data(), affine, header).to_filename(out_file)
    del img

    return out_file

temporal_filter = Node(Function(input_names=['in_file', 'TR', 'tFilter'],
                        output_names=['out_file'],
                        function=apply_temporal_filter),
               name='temp_filter')
temporal_filter.iterables = ('tFilter', filters_temporal)

In [ ]:
# Applies gaussian spatial filter as in Sengupta, Pollmann & Hanke, 2018
def gaussian_spatial_filter(in_file, sFilter, bandwidth=1):

    import nibabel as nb
    from nilearn.image import smooth_img

    ftype, fwhm = sFilter

    if ftype == 'LP':
        img = smooth_img(in_file, fwhm=fwhm)
        
    if ftype == 'HP':
        img = nb.load(in_file)
        HPF_bold = img.get_data() - smooth_img(in_file, fwhm=fwhm).get_data()
        img = nb.Nifti1Image(HPF_bold, img.get_affine())
        
    elif ftype == 'BP':
        LPF_bold_1 = smooth_img(in_file, fwhm=fwhm)
        LPF_bold_2 = smooth_img(in_file, fwhm=fwhm - bandwidth)
        BPF_bold = LPF_bold_2.get_data() - LPF_bold_1.get_data()
        img = nb.Nifti1Image(BPF_bold, LPF_bold_1.affine, LPF_bold_1.header)
        
    # Save and return output file
    out_file = in_file.replace('.nii', '_%s_%smm.nii' % (ftype, fwhm))
    img.to_filename(out_file)
    del img

    return out_file

# Spatial Band-Pass Filter
spatial_filter = Node(Function(input_names=['in_file', 'sFilter'],
                        output_names=['out_file'],
                        function=gaussian_spatial_filter),
               name='spatial_filter')
spatial_filter.iterables = ('sFilter', filters_spatial)

In [ ]:
# Computes mean image
meanimg = Node(MeanImage(dimension='T',
                         output_type='NIFTI_GZ'),
               name='meanimg')

### Create Main Workflow

**Note:** Slice time correction is applied after motion correction, as recommended by Power et al. (2017): http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0182939

In [ ]:
# Create main preprocessing workflow
mainflow = Workflow(name='mainflow')

In [ ]:
# Add nodes to workflow and connect them
mainflow.connect([(reorient, nonsteady_detection, [('out_file', 'in_file')]),
                  (reorient, nonsteady_removal, [('out_file', 'in_file')]),
                  (reorient, write_nss, [('out_file', 'in_file')]),
                  (nonsteady_detection, write_nss, [('n_volumes_to_discard', 'n_volumes')]),
                  (nonsteady_detection, nonsteady_removal, [('n_volumes_to_discard',
                                                             't_min')]),
                  (nonsteady_removal, realign, [('roi_file', 'in_files')]),
                  (realign, slicetime, [('realigned_files', 'in_files')]),
                  (getParam, slicetime, [('TR', 'time_repetition'),
                                         ('slice_order', 'slice_order'),
                                         ('nslices', 'num_slices'),
                                         ('time_acquisition', 'time_acquisition'),
                                         ]),
                  (slicetime, reset_TR, [('timecorrected_files', 'in_file')]),
                  (getParam, reset_TR, [('TR', 'TR')]),
                  (reset_TR, bet_func, [('out_file', 'in_file')]),
                  (bet_func, bet_mean, [('out_file', 'in_file')]),

                  # Coregistration
                  (coreg_pre, coreg_bbr, [('out_matrix_file', 'in_matrix_file')]),
                  (coreg_bbr, applycoreg, [('out_matrix_file', 'in_matrix_file')]),
                  (bet_mean, coreg_pre, [('out_file', 'in_file')]),
                  (bet_mean, coreg_bbr, [('out_file', 'in_file')]),
                  (bet_func, applycoreg, [('out_file', 'in_file')]),
                  
                  # Apply Temporal and Spatial Filter
                  (getParam, temporal_filter, [('TR', 'TR')]),
                  (applycoreg, temporal_filter, [('out_file', 'in_file')]),
                  (temporal_filter, spatial_filter, [('out_file', 'in_file')]),
                  (applycoreg, meanimg, [('out_file', 'in_file')]),
                  ])

## Create a subworkflow for the Confound Workflow

### Implement Nodes

In [ ]:
# Run ACompCor (based on Behzadi et al., 2007)
aCompCor = Node(ACompCor(num_components=ncomp,
                         pre_filter=False,
                         save_pre_filter=False,
                         merge_method='union',
                         components_file='compcorA.txt'),
                name='aCompCor')

In [ ]:
# Create binary mask for ACompCor (based on Behzadi et al., 2007)
def get_csf_wm_mask(in_file, wm, csf, brainmask):
    
    from nibabel import Nifti1Image, load
    from nilearn.image import threshold_img, resample_to_img
    from scipy.ndimage.morphology import binary_erosion, binary_closing

    # Create eroded WM binary mask
    thr_wm = threshold_img(wm, 0.99)
    res_wm = resample_to_img(thr_wm, in_file)
    bin_wm = threshold_img(res_wm, 0.5)
    mask_wm = binary_erosion(bin_wm.get_data(), iterations=2).astype('int8')

    # Create eroded CSF binary mask (differs from Behzadi et al., 2007)
    thr_csf = threshold_img(csf, 0.99)
    res_csf = resample_to_img(thr_csf, in_file)
    bin_csf = threshold_img(res_csf, 0.5)
    close_csf = binary_closing(bin_csf.get_data(), iterations=1)
    mask_csf = binary_erosion(close_csf, iterations=1).astype('int8')
    
    # Combine WM and CSF binary masks into one and apply brainmask
    mask_brain = load(brainmask).get_data()
    binary_mask = (((mask_wm + mask_csf) * mask_brain) > 0).astype('int8')
    out_file = in_file.replace('.nii', '_maskA.nii')
    Nifti1Image(binary_mask, res_wm.affine).to_filename(out_file)

    return out_file

acomp_masks = Node(Function(input_names=['in_file', 'wm', 'csf', 'brainmask'],
                            output_names=['out_file'],
                            function=get_csf_wm_mask),
                   name='acomp_masks')

In [ ]:
# Run TCompCor (based on Behzadi et al., 2007)
tCompCor = Node(TCompCor(num_components=ncomp,
                         percentile_threshold=0.02,
                         pre_filter=False,
                         save_pre_filter=False,
                         components_file='compcorT.txt'),
                name='tCompCor')

In [ ]:
# Create binary mask for TCompCor approach (based on Behzadi et al., 2007)
def get_brainmask(in_file):
    
    from nibabel import Nifti1Image
    from nilearn.image import mean_img
    from scipy.ndimage.morphology import binary_erosion
    
    img = mean_img(in_file)
    erod_img = binary_erosion(img.get_data()>0, iterations=1).astype('int8')
    
    out_file = in_file.replace('.nii', '_maskT.nii')
    Nifti1Image(erod_img, img.affine).to_filename(out_file)
    
    return out_file

tcomp_brainmask = Node(Function(input_names=['in_file'],
                          output_names=['out_file'],
                          function=get_brainmask),
                 name='tcomp_brainmask')

In [ ]:
# Compute FramewiseDisplacement
FD = Node(FramewiseDisplacement(parameter_source='SPM',
                                normalize=False),
          name='FD')

In [ ]:
# Compute DVARS
dvars = Node(ComputeDVARS(remove_zerovariance=True,
                          save_std=True),
           name='dvars')

In [ ]:
# Computes Friston 24-parameter model (Friston et al., 1996)
def compute_friston24(in_file):
    
    import numpy as np
    
    # Load raw motion parameters
    mp_raw = np.loadtxt(in_file)
    
    # Get motion paremter one time point before
    mp_minus1 = np.vstack((mp_raw[1:], [0] * 6))
    
    # Combine the two
    mp_combine = np.hstack((mp_raw, mp_minus1))

    # Add the square of those parameters to allow correction of nonlinear effects
    mp_friston = np.hstack((mp_combine, mp_combine**2))

    # Save friston 24-parameter model in new txt file
    out_file = in_file.replace('.txt', 'friston24.txt')
    np.savetxt(out_file, mp_friston,
               fmt='%.8f', delimiter=' ', newline='\n')
    
    return out_file

friston24 = Node(Function(input_names=['in_file'],
                          output_names=['out_file'],
                          function=compute_friston24),
                 name='friston24')

In [ ]:
# Compute average signal in total volume, in GM, in WM and in CSF
def get_average_signal(in_file, gm, wm, csf, brainmask):
    
    from scipy.stats import zscore
    from nilearn.masking import apply_mask
    from nilearn.image import threshold_img, resample_to_img, math_img, mean_img

    # Create masks for signal extraction
    res_gm = resample_to_img(threshold_img(gm, 0.99), in_file)
    bin_gm = math_img('img1>=0.5', img1=res_gm)

    res_wm = resample_to_img(threshold_img(wm, 0.99), in_file)
    bin_wm = math_img('img1>=0.5', img1=res_wm)

    res_csf = resample_to_img(threshold_img(csf, 0.99), in_file)
    bin_csf = math_img('img1>=0.5', img1=res_csf)

    res_brain = resample_to_img(brainmask, in_file)
    bin_brain = math_img('img1>=0.5', img1=res_brain)

    # Compute average signal per mask and zscore timeserie
    signal_gm = zscore(apply_mask(in_file, mask_img=bin_gm).mean(axis=-1))
    signal_wm = zscore(apply_mask(in_file, mask_img=bin_wm).mean(axis=-1))
    signal_csf = zscore(apply_mask(in_file, mask_img=bin_csf).mean(axis=-1))
    signal_brain = zscore(apply_mask(in_file, mask_img=bin_brain).mean(axis=-1))

    return [signal_brain, signal_gm, signal_wm, signal_csf]

average_signal = Node(Function(input_names=['in_file', 'gm', 'wm', 'csf', 'brainmask'],
                               output_names=['average'],
                               function=get_average_signal),
                      name='average_signal')

In [ ]:
# Combine confound parameters into one TSV file
def consolidate(FD, DVARS, par_rp, par_friston, compA, compT, average):
    
    import numpy as np
    
    conf_FD = np.array([0] + list(np.loadtxt(FD, skiprows=1)))
    conf_DVARS = np.array([1] + list(np.loadtxt(DVARS, skiprows=0)))
    conf_rp = np.loadtxt(par_rp)
    conf_friston = np.loadtxt(par_friston)
    conf_compA = np.loadtxt(compA, skiprows=1)
    conf_compT = np.loadtxt(compT, skiprows=1)
    conf_average = np.array(average)

    # Aggregate confounds
    confounds = np.hstack((conf_FD[..., None],
                           conf_DVARS[..., None],
                           conf_average.T,
                           conf_rp,
                           conf_friston,
                           conf_compA,
                           conf_compT))

    # Create header
    header = ['FD', 'DVARS']
    header += ['TV', 'GM', 'WM', 'CSF']
    header += ['Motion%02d' % (d + 1) for d in range(conf_rp.shape[1])]
    header += ['Friston%02d' % (d + 1) for d in range(conf_friston.shape[1])]
    header += ['CompA%02d' % (d + 1) for d in range(conf_compA.shape[1])]
    header += ['CompT%02d' % (d + 1) for d in range(conf_compT.shape[1])]

    # Write to file
    out_file = par_rp.replace('rp', 'confounds')
    out_file = out_file.replace('.txt', '.tsv')
    with open(out_file, 'w') as f:
        f.write('\t'.join(header) + '\n')
        for row in confounds:
            f.write('\t'.join([str(r) for r in row]) + '\n')
    
    return out_file

combine_confounds = Node(Function(input_names=['FD', 'DVARS',
                                               'par_rp', 'par_friston',
                                               'compA', 'compT', 'average'],
                                  output_names=['out_file'],
                                  function=consolidate),
                         name='combine_confounds')

### Create Confound Workflow

In [ ]:
# Create confound extraction workflow
confflow = Workflow(name='confflow')

In [ ]:
# Add nodes to workflow and connect them
confflow.connect([(acomp_masks, aCompCor, [('out_file', 'mask_files')]),
                  (tcomp_brainmask, tCompCor, [('out_file', 'mask_files')]),
                  (tcomp_brainmask, dvars, [('out_file', 'in_mask')]),
                  (tcomp_brainmask, acomp_masks, [('out_file', 'brainmask')]),

                  # Consolidate confounds
                  (FD, combine_confounds, [('out_file', 'FD')]),
                  (dvars, combine_confounds, [('out_std', 'DVARS')]),
                  (aCompCor, combine_confounds, [('components_file', 'compA')]),
                  (tCompCor, combine_confounds, [('components_file', 'compT')]),
                  (friston24, combine_confounds, [('out_file', 'par_friston')]),
                  (average_signal, combine_confounds, [('average', 'average')]),
                  ])

## Specify Input & Output Stream

In [ ]:
# Get all functional files
from bids.grabbids import BIDSLayout
layout = BIDSLayout('/data/')

In [ ]:
# Get session name if it exists
session_list = layout.get_sessions()
session_list = session_list if session_list else ['']

In [ ]:
# Get name specifier for runs per task
run_list = [(l.task, 'task-%s_run-%02d' % (l.task, int(l.run)))
            for l in layout.get(modality='func')]

# Keep only tasks that are specified in task_list
run_list = np.unique([r[1] for r in run_list if r[0] in task_list]).tolist()

In [ ]:
# Iterate over subject, session, task and run id
infosource = Node(IdentityInterface(fields=['subject_id', 'session_id', 'run_id']),
                  name='infosource')
infosource.iterables = [('subject_id', subject_list),
                        ('session_id', session_list),
                        ('run_id', run_list)]

In [ ]:
# Compute Brain Mask and Extract Brain
def create_file_path(subject_id, session_id, run_id, layout):

    from os.path import join
    
    entities = {'subject_id': subject_id,
                'run_id': run_id}
    
    # Add session id if present in dataset
    if session_id != '':
        entities['session_id'] = session_id

    # Collect input files
    in_files = []

    # Get functional image
    pattern = '/data/sub-{subject_id}[/ses-{session_id}]/func/'
    pattern += 'sub-{subject_id}[_ses-{session_id}]_{run_id}_bold.nii.gz'
    fpath = layout.build_path(entities, path_patterns=[pattern])
    in_files.append(join('/data', fpath))
    
    # Get anatomical images
    for t in ['brain.nii.gz', 'seg_gm.nii', 'seg_wm.nii', 'seg_csf.nii', 'brainmask.nii.gz']:
        pattern = 'sub-{subject_id}/sub-{subject_id}[_ses-{session_id}]_%s' % t
        fpath = layout.build_path(entities, path_patterns=[pattern])
        fpath = join('/data/derivatives/fmriflows/preproc_anat', fpath)
        in_files.append(fpath)

    return in_files

selectfiles = Node(Function(input_names=['subject_id', 'session_id',
                                         'run_id',  'layout'],
                            output_names=['func', 'brain', 'gm', 'wm', 'csf', 'brainmask'],
                            function=create_file_path),
                   name='selectfiles')
selectfiles.inputs.layout = layout

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [(
    '_run_id_%s_session_id_%s_subject_id_%s/' % (run, sess, sub),
    'sub-%s/sub-%s_ses-%s_%s_' % (sub, sub, sess, run))
    for sub in subject_list
    for sess in session_list
    for run in run_list]
substitutions += [
    ('sub-%s_%s_bold' % (sub, run), '')
    for sub in subject_list
    for run in run_list]
substitutions += [
    ('sub-%s_ses-%s_%s_bold' % (sub, sess, run), '')
    for sub in subject_list
    for sess in session_list
    for run in run_list]
substitutions += [('ar_', ''),
                  ('ras_', ''),
                  ('roi_', ''),
                  ('TR_', ''),
                  ('brain_', ''),
                  ('flirt_', ''),
                  ('tfilt', 'tFilter'),
                  ('mask_000', 'maskT'),
                  ('_roi', '_'),
                  ('__', '_'),
                  ('_.', '.'),
                  ('ses-_', ''),
                  ]
substitutions += [('%s_%smm' % (s[0], s[1]),
                   'sFilter_%s_%smm' % (s[0], s[1]))
                  for s in filters_spatial]
datasink.inputs.substitutions = substitutions

## Implement Functional Preprocessing Workflow

In [ ]:
# Create functional preprocessing workflow
preproc_func = Workflow(name='preproc_func')
preproc_func.base_dir = work_dir

# Connect input nodes to each other
preproc_func.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                                 ('session_id', 'session_id'),
                                                 ('run_id', 'run_id')])])

In [ ]:
# Add input and output nodes and connect them to the main workflow
preproc_func.connect([(infosource, mainflow, [('run_id', 'getParam.run_id')]),
                      (selectfiles, mainflow, [('func', 'reorient.in_file'),
                                               ('brain', 'coreg_pre.reference'),
                                               ('brain', 'coreg_bbr.reference'),
                                               ('wm', 'coreg_bbr.wm_seg'),
                                               ('brain', 'applycoreg.reference'),
                                               ]),
                      
                      (mainflow, datasink, [
                          ('spatial_filter.out_file', 'preproc_func.@func'),
                          ('realign.realignment_parameters', 'preproc_func.@rp_par'),
                          ('meanimg.out_file', 'preproc_func.@mean'),
                          ('write_nss.out_file', 'preproc_func.@nss')]),
                     ])

In [ ]:
# Add input and output nodes and connect them to the confound workflow
preproc_func.connect([(selectfiles, confflow, [('gm', 'average_signal.gm'),
                                               ('wm', 'average_signal.wm'),
                                               ('csf', 'average_signal.csf'),
                                               ('brainmask', 'average_signal.brainmask'),
                                               ('wm', 'acomp_masks.wm'),
                                               ('csf', 'acomp_masks.csf')]),

                      (confflow, datasink, [
                          ('tCompCor.high_variance_masks', 'preproc_func.@maskT'),
                          ('acomp_masks.out_file', 'preproc_func.@maskA'),
                          ('combine_confounds.out_file', 'preproc_func.@confound_tsv')
                      ]),
                     ])

In [ ]:
# Connect main workflow with confound workflow
preproc_func.connect([(mainflow, confflow, [
                          ('getParam.TR', 'aCompCor.repetition_time'),
                          ('applycoreg.out_file', 'aCompCor.realigned_file'),
                          ('applycoreg.out_file', 'acomp_masks.in_file'),
                          ('getParam.TR', 'tCompCor.repetition_time'),
                          ('applycoreg.out_file', 'tCompCor.realigned_file'),
                          ('applycoreg.out_file', 'tcomp_brainmask.in_file'),
                          ('applycoreg.out_file', 'average_signal.in_file'),
    
                          ('realign.realignment_parameters', 'combine_confounds.par_rp'),
                          ('realign.realignment_parameters', 'friston24.in_file'),
                          ('realign.realignment_parameters', 'FD.in_file'),
                          ('getParam.TR', 'FD.series_tr'),
                          ('applycoreg.out_file', 'dvars.in_file'),
                          ('getParam.TR', 'dvars.series_tr'),
                          ])
                     ])

## Add report nodes

In [ ]:
# Plot mean image with ACompCor and TCompCor mask ovleray
def plot_compcor_mask(sub_id, ses_id, run_id, mean, maskA, maskT):
    
    import numpy as np
    import nibabel as nb
    from matplotlib.pyplot import figure
    from nilearn.plotting import plot_anat
    from nilearn.image import coord_transform

    # Support Function to get optimal cut for visualization
    def get_cut_ids(img, axis=0):

        # Compute voxel id to cut
        idx = np.sort(img.get_data().nonzero()[axis])
        vox_id = np.linspace(idx.min(), idx.max(), num=12, endpoint=True).astype('int')
        vox_id = vox_id[2:-2]

        # Translate voxel id to image space
        if axis == 0:
            cut_ids = [int(coord_transform(r, 0, 0, img.affine)[0]) for r in vox_id]
        elif axis == 1:
            cut_ids = [int(coord_transform(0, r, 0, img.affine)[1]) for r in vox_id]
        elif axis == 2:
            cut_ids = [int(coord_transform(0, 0, r, img.affine)[2]) for r in vox_id]
        return cut_ids

    # Visualize preprocessed functional mean on subject anatomy
    def plot_mean(mean, maksA, maskT, title, out_file):
        fig = figure(figsize=(16, 8))

        for i, e in enumerate(['x', 'y', 'z']):
            ax = fig.add_subplot(3, 1, i + 1)

            display = plot_anat(mean, title=title_txt + ' - %s-axis' % e, colorbar=False,
                                display_mode=e, cut_coords=get_cut_ids(nb.load(mean), i),
                                annotate=False, axes=ax)
            display.add_overlay(maskA, cmap='plasma_r')
            display.add_overlay(maskT, cmap='winter_r')
        out_file = mean.replace('_mean.nii.gz', '_overlays.svg')
        fig.savefig(out_file, bbox_inches='tight', facecolor='black', frameon=True,
                    dpi=300, transparent=True)

    task, run = run_id.split('_')
    task = task[5:]
    run = run[4:]

    # If needed, create title for output figures
    title_txt = 'Sub: %s - Task: %s - Run: %s' % (sub_id, task, run)
    if ses_id:
        title_txt = title_txt.replace('Run', 'Sess: %s - Run' % ses_id)
    title_txt

    out_file = mean.replace('_mean.nii.gz', '_overlays.svg')

    plot_mean(mean, maskA, maskT, title_txt, out_file)
    
    return out_file

compcor_plot = Node(Function(input_names=['sub_id', 'ses_id', 'run_id',
                                          'mean', 'maskA', 'maskT'],
                          output_names=['out_file'],
                          function=plot_compcor_mask),
                 name='compcor_plot')

In [ ]:
# Plot confounds and detect outliers
def plot_confounds(confounds, outlier_thr):

    # This plotting is heavily based on MRIQC's visual reports (credit to oesteban)
    import numpy as np
    import pandas as pd
    from matplotlib.backends.backend_pdf import FigureCanvasPdf as FigureCanvas
    import seaborn as sns
    sns.set(style="darkgrid")
    from matplotlib import pyplot as plt
    from matplotlib.gridspec import GridSpec

    def plot_timeseries(dataframe, elements, out_file, outlier_thr=None):

        # Number of rows to plot
        n_rows = len(elements)

        # Create canvas
        fig = plt.Figure(figsize=(16, 2 * n_rows))
        FigureCanvas(fig)
        grid = GridSpec(n_rows, 2, width_ratios=[7, 1])

        # Specify color palette to use
        colors = sns.husl_palette(n_rows)

        # To collect possible outlier indices
        outlier_idx = []

        # Plot timeseries (and detect outliers, if specified)
        for i, e in enumerate(elements):

            # Extract timeserie values
            data = dataframe[e].values

            # Plot timeserie
            ax = fig.add_subplot(grid[i, :-1])
            ax.plot(data, color=colors[i])
            ax.set_xlim((0, len(data)))
            ax.set_ylabel(e)
            ylim = ax.get_ylim()

            # Detect and plot outliers if threshold is specified
            if outlier_thr:
                outlier_id = np.where(np.abs(data)>=outlier_thr[i])[0]
                outlier_idx += list(outlier_id)
                ax.vlines(outlier_id, ylim[0], ylim[1])

            # Plot observation distribution
            ax = fig.add_subplot(grid[i, -1])
            sns.distplot(data, vertical=True, ax=ax, color=colors[i])
            ax.set_ylim(ylim)
            
        fig.savefig(out_file)

        return np.unique(outlier_idx)

    # Load confounds table
    df = pd.read_table(confounds)

    # Aggregate output plots
    out_plots = []
    
    # Plot main confounds
    elements = ['FD', 'DVARS', 'TV', 'GM', 'WM', 'CSF']
    out_file = confounds.replace('.tsv', '_main.svg')
    out_plots.append(out_file)
    outliers = plot_timeseries(df, elements, out_file, outlier_thr)
    
    # Save outlier indices to textfile
    outlier_filename = confounds.replace('.tsv', '_outliers.txt')
    np.savetxt(outlier_filename, outliers, fmt='%d')

    # Plot Motion Paramters
    elements = [k for k in df.keys() if 'Motion' in k]
    out_file = confounds.replace('.tsv', '_motion.svg')
    out_plots.append(out_file)
    plot_timeseries(df, elements, out_file)

    # Plot CompCor components
    for comp in ['A', 'T']:
        elements = [k for k in df.keys() if 'Comp%s' % comp in k]
        out_file = confounds.replace('.tsv', '_comp%s.svg' % comp)
        out_plots.append(out_file)
        plot_timeseries(df, elements, out_file)
    
    return [outlier_filename] + out_plots

confound_inspection = Node(Function(input_names=['confounds', 'outlier_thr'],
                                    output_names=['out_file', 'plot_main', 'plot_motion',
                                                  'plot_compA', 'plot_compT'],
                                    function=plot_confounds),
                           name='confound_inspection')
confound_inspection.inputs.outlier_thr = outlier_thr

In [ ]:
# Update report
def write_report(sub_id, ses_id, run_list, cor_plot, conf_plot):

    # Get list of all individual tasks and runs
    func_idx = [run_id[5:].split('_run-') for run_id in run_list]

    # Load template for functional preprocessing output
    with open('/templates/report_template_func.html', 'r') as report:
        func_temp = report.read()

    # Get filename of html report
    if ses_id:
        html_file = '/data/derivatives/fmriflows/sub-%s_ses-%s.html' % (sub_id, ses_id)
    else:
        html_file = '/data/derivatives/fmriflows/sub-%s.html' % sub_id

    # Old template placeholder
    func_key = '<p>The functional preprocessing pipeline hasn\'t been run yet.</p>'
    
    # Add new content to report
    with open(html_file, 'r') as report:
        txt = report.read()
        
        # Reset report with functional preprocessing template
        cut_start = txt.find('Functional Preprocessing</a></h2>') + 33
        cut_stop = txt.find('<!-- Section: 1st-Level Univariate Results-->')
        txt = txt[:cut_start] + func_key + txt[cut_stop:]

        txt_amendment = ''

        # Go through the tasks and runs
        for task_id, run_id in func_idx:

            func_txt = func_temp.replace('sub-placeholder', 'sub-%s' % sub_id)
            func_txt = func_txt.replace('task-placeholder', 'task-%s' % task_id)
            func_txt = func_txt.replace('run-placeholder', 'run-%s' % run_id)

            # Add session suffix if present
            if ses_id:
                func_txt = func_txt.replace('ses-placeholder', 'ses-%s' % ses_id)
            else:
                func_txt = func_txt.replace('ses-placeholder', '')
                func_txt = func_txt.replace('__', '_')

            txt_amendment += func_txt
 
    # Add pipeline graphs
    txt_amendment += '<h3 class="h3" style="position:left;font-weight:bold">Graph of'
    txt_amendment += ' Functional Preprocessing pipeline</h3>\n    <object data="preproc_func/graph.svg"'
    txt_amendment += ' type="image/svg+xml" style="width:100%"></object>\n  '
    txt_amendment += ' <object data="preproc_func/graph_detailed.svg" type="image/svg+xml"'
    txt_amendment += ' style="width:100%"></object>\n'

    # Insert functional preprocessing report
    txt = txt.replace(func_key, txt_amendment)

    # Overwrite previous report
    with open(html_file, 'w') as report:
        report.writelines(txt)

create_report = MapNode(Function(input_names=['sub_id', 'ses_id', 'run_list',
                                              'cor_plot', 'conf_plot'],
                                 output_names=['out_file'],
                                 function=write_report),
                     name='create_report', iterfield=['cor_plot', 'conf_plot'])
create_report.inputs.run_list = run_list

In [ ]:
preproc_func.connect([(infosource, compcor_plot, [('subject_id', 'sub_id'),
                                                  ('session_id', 'ses_id'),
                                                  ('run_id', 'run_id')]),
                      (mainflow, compcor_plot, [('meanimg.out_file', 'mean')]),
                      (confflow, compcor_plot, [
                          ('tCompCor.high_variance_masks', 'maskT'),
                          ('acomp_masks.out_file', 'maskA')
                      ]),
                      (compcor_plot, datasink, [('out_file', 'preproc_func.@compcor_plot')]),

                      (confflow, confound_inspection, [
                          ('combine_confounds.out_file', 'confounds'),
                      ]),
                      (confound_inspection, datasink, [('out_file', 'preproc_func.@conf_inspect'),
                                                       ('plot_main', 'preproc_func.@conf_main'),
                                                       ('plot_motion', 'preproc_func.@conf_motion'),
                                                       ('plot_compA', 'preproc_func.@conf_compA'),
                                                       ('plot_compT', 'preproc_func.@conf_compT')]),

                      (infosource, create_report, [('subject_id', 'sub_id'),
                                                   ('session_id', 'ses_id')]),
                      (compcor_plot, create_report, [('out_file', 'cor_plot')]),
                      (confound_inspection, create_report, [('plot_main', 'conf_plot')]),
                     ])

## Visualize Workflow

In [ ]:
# Create preproc_func output graph
preproc_func.write_graph(graph2use='colored', format='svg', simple_form=True)

# Visualize the graph
from IPython.display import SVG
SVG(filename=opj(preproc_func.base_dir, 'preproc_func', 'graph.svg'))

# Run Workflow

In [ ]:
# Run the workflow in parallel mode
preproc_func.run(plugin='MultiProc', plugin_args={'n_procs' : n_proc})

In [ ]:
# Save workflow graph visualizations in datasink
preproc_func.write_graph(graph2use='flat', format='svg', simple_form=True)
preproc_func.write_graph(graph2use='colored', format='svg', simple_form=True)

from shutil import copyfile
copyfile(opj(preproc_func.base_dir, 'preproc_func', 'graph.svg'),
         opj(exp_dir, out_dir, 'preproc_func', 'graph.svg'))
copyfile(opj(preproc_func.base_dir, 'preproc_func', 'graph_detailed.svg'),
         opj(exp_dir, out_dir, 'preproc_func', 'graph_detailed.svg'));